In [2]:
import pandas as pd
from pandas import Series, DataFrame
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
import numpy as np
folder = "/data/Peptides/dendrimers/T7-GAanalogs/"
from Levenshtein import distance as lev_dist
import tmap as tm
from map4 import MAP4Calculator
import random
import torch.nn as nn
import torch
import time
import math
import pickle
import pandas as pd
import os
from pandas import Series, DataFrame
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, matthews_corrcoef, f1_score, precision_score, recall_score
import random
import pickle
from rdkit.Chem import rdchem, Lipinski
from rdkit import Chem
from rdkit.Chem.rdmolfiles import MolFromFASTA, MolToSmiles, MolFromSmiles
from sklearn.model_selection import GridSearchCV
import numpy as np
import torch.optim as optim
import joblib
from scipy.stats import rankdata
from faerun import Faerun
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sys
from scipy import stats
from matplotlib.colors import LinearSegmentedColormap, BoundaryNorm
from sklearn.neighbors import NearestNeighbors
from tqdm.notebook import tqdm


INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
elements = ["K","L",""]
branching_elements = ["B"]
enumeration = {}
enumeration[1] = [e for e in elements]

for i in range (2,16):
    if i % 4 == 0:
        elements_to_append = branching_elements
    else:
        elements_to_append = elements
    enumeration[i] = set()
    for e in elements_to_append:
        for previous_enumeration in enumeration[i-1]:
            enumeration[i].add( previous_enumeration + e )


In [4]:
len(enumeration[15])

50625

In [5]:
KL_enum = enumeration[15]

In [6]:
lib = pd.read_csv("/data/Peptides/dendrimers/T25_analogs/database/52k.mxfp", sep=" ", header=None)

In [7]:
lib.columns=["SMILES","seq", "MXFP", "prop"]

In [8]:
lib["seq2"] = lib.seq.map(lambda x: x.replace("(", "").replace(")","").replace("8K","B").replace("4K","B").replace("2K","B"))

In [9]:
def is_in_KL_enum(seq):
    if seq[0] == "K" and "B"+seq[1:] in KL_enum:
        return True
    elif seq in KL_enum:
        return True
    else:
        return False

In [10]:
lib["in_50k"] = lib.seq2.map(is_in_KL_enum)

In [11]:
lib.query("in_50k == True")

,SMILES,seq,MXFP,prop,seq2,in_50k
0,CC(C)CC(NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(...,(KLL)8(KLLL)4(KLLL)2KLLL,4;4;6;7;8;9;10;12;15;18;23;30;39;47;55;67;86;1...,432;260;0;53;67;16;1,KLLBLLLBLLLBLLL,True
1,CC(C)CC(NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(...,(KKL)8(KLLL)4(KLLL)2KLLL,4;4;6;7;8;9;10;12;15;18;23;29;38;47;55;67;85;1...,440;260;0;53;75;24;1,KKLBLLLBLLLBLLL,True
2,CC(C)CC(NC(=O)C(CC(C)C)NC(=O)C(CCCCNC(=O)C(CC(...,(KLL)8(KLLL)4(KLLL)2KLL,4;4;6;7;8;9;10;12;15;18;23;30;39;47;55;66;84;1...,424;255;0;52;66;16;1,KLLBLLLBLLLBLL,True
3,CC(C)CC(NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(...,(KLL)8(KLLL)4(KLL)2KLLL,4;4;6;7;8;10;10;12;15;18;23;30;39;48;57;70;90;...,416;250;0;51;65;16;1,KLLBLLLBLLBLLL,True
4,CC(C)CC(NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(...,(KLL)8(KLL)4(KLLL)2KLLL,5;4;6;7;8;10;10;12;15;19;24;31;40;50;60;74;92;...,400;240;0;49;63;16;1,KLLBLLBLLLBLLL,True
...,...,...,...,...,...,...
52520,[NH3+]CCCCC([NH3+])C(=O)NC(CCCC[NH3+])C(=O)NC(...,(KKK)4(KKKK)2K,6;6;8;9;10;12;14;16;21;26;31;37;46;58;67;67;65...,226;125;0;26;50;26;1,KKKBKKKB,True
52525,[NH3+]CCCCC([NH3+])C(=O)NC(CCCC[NH3+])C(=O)NC(...,(KKKK)4(K)2K,6;6;8;9;11;12;14;17;21;26;31;36;43;53;70;89;10...,208;115;0;24;46;24;1,KKKKBB,True
52526,[NH3+]CCCCC([NH3+])C(=O)NC(CCCC[NH3+])C(=O)NC(...,(KKKK)4(KK)2K,6;6;8;9;10;12;14;16;21;26;31;37;44;53;66;79;88...,226;125;0;26;50;26;1,KKKKBKB,True
52527,[NH3+]CCCCC([NH3+])C(=O)NC(CCCC[NH3+])C(=O)NC(...,(KKKK)4(KKK)2K,6;6;7;9;10;12;13;16;20;25;30;36;44;54;65;74;78...,244;135;0;28;54;28;1,KKKKBKKB,True


In [12]:
lib.query("in_50k == False")

,SMILES,seq,MXFP,prop,seq2,in_50k
742,CC(C)CC(NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(...,(LLL)4(KLLL)2K,6;6;9;10;12;14;14;17;21;25;30;36;45;56;62;62;6...,208;125;0;26;32;8;1,LLLBLLLB,False
743,CC(C)CC(NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(...,(KLLL)2KLLL,10;9;13;14;17;19;20;24;29;33;38;45;54;62;58;38...,100;60;0;13;15;4;1,KLLLBLLL,False
1196,CC(C)CC(NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(...,(KKLL)2KLLL,9;9;12;14;16;19;20;23;29;33;38;45;54;61;59;41;...,102;60;0;13;17;6;1,KKLLBLLL,False
1241,CC(C)CC(NC(=O)C(CC(C)C)NC(=O)C(CCCCNC(=O)C(CC(...,(LLL)4(KLL)2K,7;7;9;10;12;14;15;17;22;25;30;37;46;56;62;65;6...,192;115;0;24;30;8;1,LLLBLLB,False
1242,CC(C)CC(NC(=O)C(CC(C)C)NC(=O)C(CCCCNC(=O)C(CC(...,(KLLL)2KLL,10;10;13;15;18;20;20;24;29;33;39;45;52;56;49;3...,92;55;0;12;14;4;1,KLLLBLL,False
...,...,...,...,...,...,...
52521,[NH3+]CCCCC([NH3+])C(=O)NC(CCCC[NH3+])C(=O)NC(...,(KKKK)2KKKK,9;9;11;13;15;17;19;23;28;34;40;47;56;65;65;46;...,109;60;0;13;24;13;1,KKKKBKKK,False
52522,[NH3+]CCCCC([NH3+])C(=O)NC(CCCC[NH3+])C(=O)NC(...,(KKKK)2KKK,10;9;12;14;15;18;20;23;29;35;41;47;54;60;56;38...,100;55;0;12;22;12;1,KKKKBKK,False
52523,[NH3+]CCCCC([NH3+])C(=O)NC(CCCC[NH3+])C(=O)NC(...,(KKKK)2KK,10;10;12;14;16;18;20;24;29;35;40;44;48;51;48;3...,91;50;0;11;20;11;1,KKKKBK,False
52524,[NH3+]CCCCC([NH3+])C(=O)NC(CCCC[NH3+])C(=O)NC(...,(KKK)2K,11;10;13;15;17;19;21;24;30;34;37;38;38;42;44;3...,82;45;0;10;18;10;1,KKKB,False


In [13]:
def mxfp_(mxfp):
    mxfp_list = mxfp.split(";")
    mxfp_int = np.array(list(map(lambda x: int(x), mxfp_list)))
    return mxfp_int

In [14]:
lib["MXFP_"] = lib.MXFP.map(mxfp_)

In [15]:
len(lib)

52530

In [19]:
df  = lib.query("in_50k == True").copy()

In [20]:
MAP4 = MAP4Calculator(dimensions=1024)
def calc_map4(SMILES):
    mol = Chem.MolFromSmiles(SMILES)
    map4 = MAP4.calculate(mol)
    return np.array(map4)

In [21]:
df["MAP4"] = df.SMILES.parallel_map(calc_map4)

In [22]:
if not os.path.exists("/data/Peptides/dendrimers/T25_analogs/database/LSHforest"):
    fps = df["MAP4"].tolist()
    fps_ = []

    for fp in fps:
        fp_ = tm.VectorUint(fp)
        fps_.append(fp_)

    lf = tm.LSHForest(1024, 32)
    lf.batch_add(fps_)

    lf.index()
    lf.store("/data/Peptides/dendrimers/T25_analogs/database/LSHforest")
else:
    lf = tm.LSHForest(1024, 32)
    lf.restore("/data/Peptides/dendrimers/T25_analogs/database/LSHforest")

In [33]:
if not os.path.exists("/data/Peptides/dendrimers/T25_analogs/database/tm_layout_.pkl"):
    cfg = tm.LayoutConfiguration()

    # config
    cfg.node_size = 1 / 40
    cfg.mmm_repeats = 2
    cfg.sl_extra_scaling_steps = 50
    cfg.k = 60
    cfg.sl_scaling_type = tm.RelativeToAvgLength
    # Compute the layout

    x_, y_, s, t, gp = tm.layout_from_lsh_forest(lf, cfg)
    tm_layout = {"x": list(x_), "y": list(y_), "s" : list(s), "t" : list(t)}
    joblib.dump(tm_layout, "/data/Peptides/dendrimers/T25_analogs/database/tm_layout.pkl")
else:
    tm_layout = joblib.load("/data/Peptides/dendrimers/T25_analogs/database/tm_layout.pkl")

In [ ]:
if not os.path.exists("/data/Peptides/dendrimers/T25_analogs/database/edgelist_mxfp_.pkl"):
    mxfps_ = df["MXFP"].tolist()
    mxfps = []

    for mxfp_ in mxfps_:
        mxfp = np.array(list(map(int, mxfp_.split(';'))))
        mxfps.append(mxfp)    

    knn = 60

    knn_search = NearestNeighbors(n_neighbors=knn, radius=1.0, algorithm='auto', leaf_size=30, metric='manhattan', p=2, metric_params=None, n_jobs=None)
    knn_search.fit(np.array(mxfps))

    edge_list = []

    for i in tqdm(range(len(mxfps))):
        dists, idxs = knn_search.kneighbors(mxfps[i].reshape(1,-1))
        for j in range(knn):
            edge_list.append([i, idxs[0,j], dists[0,j]])
    joblib.dump(edge_list, "/data/Peptides/dendrimers/T25_analogs/database/edgelist_mxfp.pkl")
else:
    edge_list = joblib.load("/data/Peptides/dendrimers/T25_analogs/database/edgelist_mxfp.pkl")

In [ ]:
if not os.path.exists("/data/Peptides/dendrimers/T25_analogs/database/tm_layout_.pkl"):
    # Compute the layout
    x_, y_, s, t, gp = tm.layout_from_edge_list(len(edge_list), edge_list)
    tm_layout_el = {"x": list(x_), "y": list(y_), "s" : list(s), "t" : list(t)}
    joblib.dump(tm_layout_el, "/data/Peptides/dendrimers/T25_analogs/database/tm_layout.pkl")
else:
    tm_layout_el = joblib.load("/data/Peptides/dendrimers/T25_analogs/database/tm_layout.pkl")

In [26]:
relevant_2021 = pd.read_csv("/data/Peptides/dendrimers/T25_analogs/all.smi", sep=" ", header=None)
relevant_2021.columns=["SMILES", "seq"]
relevant = relevant_2021.seq.to_list()
relevant_2018 = pd.read_csv("/data/Peptides/dendrimers/G3KL-GAanalogs/actives-from-papers", sep=" ", header=None)
relevant_2018.columns=["name", "seq"]
relevant_old = relevant_2018.seq.to_list()

In [27]:
import re
def positive(seq):
    pos = 0
    gens = re.split('1|2|3|4', seq)[::-1]
    for i, gen in enumerate(gens):
        j = 0
        if i == 0:
            j = 1
        else:
            j = 2**i
        pos += (gen.count("K") + gen.count("R"))*j
    return pos

def hydrophobic(seq):
    return seq.count("A") + seq.count("V") + seq.count("L") + seq.count("I") + seq.count("F") + seq.count("M") + seq.count("W") + seq.count("C") 

def pos_hydro_ratio(row):
    if row.hydrophobic == 0:
        return row.positive
    return round(row.positive/row.hydrophobic, 2)

def negative(seq):
    return seq.count("E") + seq.count("D")

def glu_first(seq):
    first = re.split('1|2|3|4', seq)[-2]
    return first.count("E")

def glu_core(seq):
    core = re.split('1|2|3|4', seq)[-1]
    return core.count("E")

def glu(row):
    if row.glu_core == 1:
        return 1
    elif row.glu_first == 1:
        return 2
    else:
        return 0


def generations(seq):
    seq = seq[1:]
    if seq.count("1") + seq.count("2") + seq.count("3") + seq.count("4") == 2:
        return 0
    else:
        return 1
    
def HAC(prop):
    return int(prop.split(";")[0])

def isSelected(seq):
    if seq in relevant:
        return 2
    elif seq in relevant_old:
        return 1
    else:
        return 0

mxfp_query = "5;5;7;8;9;10;11;13;17;21;27;36;44;52;64;82;100;111;104;92;61;14;0;0;0;0;0;0;0;0;0;7;5;5;4;3;4;8;10;13;16;18;26;34;41;47;62;77;86;82;72;50;12;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;16;0;0;0;0;10;1;3;8;7;12;14;15;18;24;31;37;40;34;27;14;1;0;0;0;0;0;0;0;0;0;13;0;0;8;0;1;9;4;6;9;10;12;17;21;25;31;39;44;44;43;33;10;0;0;0;0;0;0;0;0;0;21;0;0;0;0;1;6;0;0;0;0;2;5;11;11;7;11;23;29;30;46;29;2;0;0;0;0;0;0;0;0;100;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0"
b = np.array(list(map(int, mxfp_query.split(';'))))
def calc_CBD(mxfp):
    a = np.array(list(map(int, mxfp.split(';'))))
    return sum(abs(a-b))

In [28]:
df["positive"] = df.seq.map(positive)
df["negative"] = df.seq.map(negative)
df["glu_core"] = df.seq.map(glu_core)
df["glu_first"] = df.seq.map(glu_first)
df["hydrophobic"] = df.seq.map(hydrophobic)
df["pos_hydro_ratio"] = df.apply(pos_hydro_ratio, 1)
df["generations"] =  df.seq.map(generations)
df["hac"] = df.prop.map(HAC)
df["selected"] = df.seq.map(isSelected)
df["CBD"] = df.MXFP.map(calc_CBD)

In [29]:
tmp = df.copy()
idx = tmp[tmp.pos_hydro_ratio >= 4].index
tmp.loc[idx, "pos_hydro_ratio"]=4

idx = tmp[tmp["positive"] >= 20].index
tmp.loc[idx, "positive"] = 20

HAC = df.hac.tolist()
cbd = df.CBD.tolist()
cbd_r = rankdata(cbd)
seq = df.seq.tolist()
SMILES = df.SMILES.tolist()
selected = df.selected.tolist()
negative =  df.negative.tolist()
positive =  tmp.positive.tolist()
generations = df.generations.tolist()
p_h_ratio =  tmp.pos_hydro_ratio.tolist()


custom_cmap = ListedColormap(['lightgray', 'magenta', "lime"], name="custom")

custom_cmap2 = ListedColormap(["gray", "sienna", "peru", "teal", "gold", "blue", "yellow", "cyan", "red", "orange", "tomato", "aquamarine", "orchid",\
                              "pink", "purple", "green", "lime", "white", 'lightgray', 'magenta', "cornflowerblue"], name="custom2")

custom_cmap3 = ListedColormap(['magenta', "lime"], name="custom3")

groups4 = ["0 - No", "1 - 2018", "2 - 2021"]
labels_groups4, groups4 = Faerun.create_categories(groups4)

groups2 = ["00-T7", "01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20"]
labels_groups2, groups2 = Faerun.create_categories(groups2)

groups3 = ["2nd generation dendrimer", "3rd generation dendrimer"]
labels_groups3, groups3 = Faerun.create_categories(groups3)

groups = ["0 - No", "1 - Yes"]
labels_groups, groups = Faerun.create_categories(groups)

groups5 = ["0 - Not present", "1 - Present in the core", "2 - Present in the first generation"]
labels_groups5, groups5 = Faerun.create_categories(groups5)

In [ ]:
faerun = Faerun(view="front", coords=False, title='MXFP_T25analogs')#, clear_color="#ffffff",)
faerun.add_scatter("T25analogs",{"x": tm.VectorFloat(tm_layout["x"]), "y": tm.VectorFloat(tm_layout["y"]),\
                                "c": [HAC, cbd_r, selected, positive, p_h_ratio, generations],\
                          "labels": seq}, has_legend=True, \
    colormap=["rainbow", "rainbow_r", custom_cmap, "rainbow","rainbow", custom_cmap3], \
        point_scale=1, categorical=[False, False, True, False, False, True],\
        series_title=["HAC", "CBD (rank)", "selection", "positive residues count", \
                      "positive/hydrophobic residues ratio", "dendrimer structure"], \
            max_legend_label=[str(max(HAC)),str(max(cbd)), None, None, ">=20", None, None, ">= 3", None],\
                   min_legend_label=[str(min(HAC)), str(min(cbd)), None, None, str(min(positive)), None, None, str(min(p_h_ratio)), None],\
                    legend_labels=[None, None, labels_groups2 ,labels_groups4, None, labels_groups5, labels_groups, None, labels_groups3])


faerun.add_tree("T25analogs_tree", {"from": tm.VectorUint(tm_layout["s"]), "to": tm.VectorUint(tm_layout["t"])}, point_helper="T25analogs", color="aaaaaa")
faerun.plot('MXFP_T25analogs')

In [ ]:
faerun = Faerun(view="front", coords=False, title='MXFP_T25analogs')#, clear_color="#ffffff",)
faerun.add_scatter("T25analogs",{"x": tm.VectorFloat(tm_layout_el["x"]), "y": tm.VectorFloat(tm_layout_el["y"]),\
                                "c": [HAC, cbd_r, selected, positive, p_h_ratio, generations],\
                          "labels": seq}, has_legend=True, \
    colormap=["rainbow", "rainbow_r", custom_cmap, "rainbow","rainbow", custom_cmap3], \
        point_scale=1, categorical=[False, False, True, False, False, True],\
        series_title=["HAC", "CBD (rank)", "selection", "positive residues count", \
                      "positive/hydrophobic residues ratio", "dendrimer structure"], \
            max_legend_label=[str(max(HAC)),str(max(cbd)), None, None, ">=20", None, None, ">= 3", None],\
                   min_legend_label=[str(min(HAC)), str(min(cbd)), None, None, str(min(positive)), None, None, str(min(p_h_ratio)), None],\
                    legend_labels=[None, None, labels_groups2 ,labels_groups4, None, labels_groups5, labels_groups, None, labels_groups3])


faerun.add_tree("T25analogs_tree", {"from": tm.VectorUint(tm_layout_el["s"]), "to": tm.VectorUint(tm_layout_el["t"])}, point_helper="T25analogs", color="aaaaaa")
faerun.plot('MXFP_T25analogs')

In [31]:
len(tm_layout["x"])

50625